Problem 1: Production economy and CO2 taxation

In [40]:
from types import SimpleNamespace
import numpy as np
from scipy.optimize import minimize, root
import pandas as pd
from scipy.optimize import differential_evolution

In [2]:
par = SimpleNamespace()
# firms
par.A = 1.0
par.gamma = 0.5
# households
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
# government
par.tau = 0.0
par.T = 0.0
# Question 3
par.kappa = 0.1

#Set w = 1 as numearie
par.w = 1

Define model

In [3]:

def l1_star(par, p1):
    return (p1 * par.A * par.gamma/par.w)**(1 / (1 - par.gamma)) 

def l2_star(par, p2):
    return (p2 * par.A * par.gamma/par.w)**(1 / (1 - par.gamma))

def y1_star(par, l1):
    return par.A * (l1_star(par,p1))**par.gamma

def y2_star(par, l2):
    return par.A * (l2_star(par,p2))**par.gamma

def pi1_star(par, p1):
    return ((1 - par.gamma) / par.gamma) * par.w * (p1 * par.A * par.gamma / par.w)**(1 / (1 - par.gamma))

def pi2_star(par, p2):
    return ((1 - par.gamma) / par.gamma) * par.w * (p2 * par.A * par.gamma / par.w)**(1 / (1 - par.gamma))

def c1(l, par, p1, p2):
    return par.alpha * ((par.w * l + par.T + pi1_star(par, p1) + pi2_star(par, p2)) / p1)

def c2(l, par, p1, p2):
    return (1 - par.alpha) * ((par.w * l + par.T + pi1_star(par, p1) + pi2_star(par, p2)) / (p2 + par.tau))

def l_star(par, p1, p2, initial_guess=0.1):
    def negative_objective(l):
        c1_val = c1(l, par, p1, p2)
        c2_val = c2(l, par, p1, p2)
        if c1_val <= 0 or c2_val <= 0:
            return np.inf  # Penalize invalid solutions
        return -(np.log((c1_val)**par.alpha * (c2_val)**(1 - par.alpha)) - par.nu * l**(1 + par.epsilon) / (1 + par.epsilon))

    result = minimize(negative_objective, initial_guess, method='BFGS')

    # Extract the optimal value of l
    l_star_value = result.x[0]

    return l_star_value


def c1_star(l_star_value, par, p1, p2):
    return c1(l_star_value, par, p1, p2)

def c2_star(l_star_value, par, p1, p2):
    return c2(l_star_value, par, p1, p2)



Define excess demand

In [4]:
def excessdemand_labor(par, p1, p2):
    l1 = l1_star(par, p1)
    l2 = l2_star(par, p2)
    l_opt = l_star(par, p1, p2)
    return l1 + l2 - l_opt


def excessdemand_goodmarket1(par, p1, p2):
    l1 = l1_star(par, p1)
    y1 = y1_star(par, l1)
    l_opt = l_star(par, p1, p2)
    c1_opt = c1_star(l_opt, par, p1, p2)
    return y1 - c1_opt

def excessdemand_goodmarket2(par, p1, p2):
    l2 = l2_star(par, p2)
    y2 = y2_star(par, l2)
    l_opt = l_star(par, p1, p2)
    c2_opt = c2_star(l_opt, par, p1, p2)
    return y2 - c2_opt

Find the excess demand for different p1 and p2 values

In [ ]:
# Define ranges for p1 and p2
p1_values = np.linspace(0.1, 2.0, 10)
p2_values = np.linspace(0.1, 2.0, 10)

# Initialize lists to store results
results = []

# Compute excess demands for each combination of p1 and p2
for p1 in p1_values:
    for p2 in p2_values:
        excess_labor = excessdemand_labor(par, p1, p2)
        excess_goodsmarket1 = excessdemand_goodmarket1(par, p1, p2)
        excess_goodsmarket2 = excessdemand_goodmarket2(par, p1, p2)
        results.append((p1, p2, excess_labor, excess_goodsmarket1, excess_goodsmarket2))

# Create a DataFrame for better readability
df_results = pd.DataFrame(results, columns=['p1', 'p2', 'Excess Labor', 'Excess Goods Market 1', 'Excess Goods Market 2'])

# Display the DataFrame
pd.set_option('display.float_format', lambda x: '%.4f' % x)
print(df_results)

**Question 2: Find p1 and p2**

In [ ]:
# Define the objective function for finding equilibrium prices
def objectivenew(vars, par):
    p1, p2 = vars
    excess_labor = excessdemand_labor(par, p1, p2)
    excess_goodsmarket2 = excessdemand_goodmarket2(par, p1, p2)
    return excess_labor**2 + excess_goodsmarket2**2

# List initial guesses in the space to see which would be best
initial_guesses = [
    [0.5, 0.5],
    [1, 1],
    [1.5, 1.5],
    [0.1, 0.1],
    [2.0, 2.0]
]

best_result = None
best_objective_value = np.inf
best_initial_guess = None


# Try initial guesses and find the best result with lowest objective value
for initial_guess in initial_guesses:
    result = minimize(objectivenew, initial_guess, args=(par,), bounds=[(0.1, 2.0), (0.1, 2.0)])
    if result.fun < best_objective_value:
        best_result = result
        best_objective_value = result.fun
        best_initial_guess = initial_guess

equilibrium_p1, equilibrium_p2 = best_result.x

#Print results for equilibrium prices and the initial guess that gave the best result
print(f"Equilibrium p1: {equilibrium_p1:.4f}, Equilibrium p2: {equilibrium_p2:.4f}")
print(f"Best initial guess: {best_initial_guess}")

**Question 3: What values of tau and T should the government choose to maximize SWF?**

In [24]:

def U():

    def negative_utility(c1, c2, l, p1, p2, par):
        consumption = log(c1(par, l, p1, p2)**par.alpha*c2(par, l, p1,p2)**(1-par.alpha))
        work = par.nu*l**(1-par.epsilon)/(1+par.epsilon)
        u = -(consumption - work)

    # Define the budget constraint
    constraint = p1 * c1 + (p2 + tau) * c2 - (w * ell + T + pi1_star + pi2_star)

    # Initial guess
    x0 = 1S

    # Minimize the objective function with the constraint
    result = minimize(objective, x0, constraints={'type': 'eq', 'fun': constraint, 'args': (params['p1'], params['p2'], params['w'], params['tau'], params['T'], params['alpha'], params['nu'], params['epsilon'], params['pi1_star'], params['pi2_star'])})

    # Print the results
    print("Optimization Result:", result)
    print(f"c1: {result.x[0]}, c2: {result.x[1]}, ell: {result.x[2]}")




def SWF(U, par, y2_star):

    return U - par.kappa * y2_star


SyntaxError: invalid decimal literal (4110019734.py, line 12)

mangler